This notebook is based on https://github.com/dmpierre/air-ccs/blob/main/sage/air-to-ccs.ipynb. This notebook implements
- Plonk-to-CCS conversion algorithm, found in the [CCS paper](https://eprint.iacr.org/2023/552), Section 2.2.
- A PoC of incorporating Halo2 lookup into CCS+.
[The explanation of math behind the algorithm is here](https://hackmd.io/@pnyda/SkG0qaUuA)


# The part where we derive c, M, S from custom gates

In [15]:
p = 21888242871839275222246405745257275088696311157297823662689037894645226208583
F = GF(p)
P.<a, b, c, d> = F[]

# Define letters
t = 4
m = 3
n = 13

# Let's say we have 4 advice columns. We constrain muladd.
g = a * b + c - d

# Advice columns
table = [[2, 1, 6], [3, 2, 7], [4, 3, 3], [10, 5, 45]]

# First we generate Z by flattening the table
z = vector(F, table[0] + table[1] + table[2] + table[3] + [1])
print(z)

# Generate M
# This method only works for this gate. How do I generalize this and make it support arbitrary gates? such as ones that refer to neighboring rows?
matrices = [matrix(F, m, n) for i in range(t)]
for (i, column) in enumerate(table):
    for (j, cell) in enumerate(column):
        matrices[i][j, i * len(column) + j] = 1

print(matrices[0])
print(matrices[1])
print(matrices[2])
print(matrices[3])

(2, 1, 6, 3, 2, 7, 4, 3, 3, 10, 5, 45, 1)
[1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0]
